# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [1]:
import networkx
from urllib.request import urlopen
homer = urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')
homer = [line.decode('utf-8') for line in homer]

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

Next implement a function to read in the edges from the input file.

In [2]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    nodes=[] # Declaring an empty list to which all nodes would be added
    node=""
    for line in gfile:
        if (not(':' in line) and not('*' in line) and not(len(line.strip()) == 0)): #Empty lines and lines with '*', ':' don't have nodes in them.
            for character in line:
                if(character==" "): #Each of the nodes have muliple characters and hence they are added to the list when any empty character is found
                    nodes.append(node) # Appending the node to our Nodes List
                    node=""
                    break
                else:
                    node=node + character  #Appending character of the node name          
    return nodes

In [3]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    edges=[]# Declaring an empty list to which all edges in tuple form would be added
    for line in gfile:
        if((':' in line) and not('*' in line) and not(len(line.strip()) == 0)):#Empty lines and lines with '*' don't have edges in them. Aldo, lines having edges in them must have ':' in it.
            #Splitting the line by ':' and taking the index 1 element that is everything after ':' as there is only one ':' in it. 
            #Further that part is splitted by ';' as edges interacting groups are seperated in such a way
            for setEdge in line.split(':')[1].split(';'): 
                # Each interacting component in the group are seperated by ',' and hence we split them by ','
                edgeList = setEdge.split(',')
                # Each node in the interacting group interacts with every other node in the group and hence for creating the edges we run 2 for loops.
                for i in range(len(edgeList)):
                    for j in range(i+1,len(edgeList)):
                        edges.append(tuple([edgeList[i].strip(),edgeList[j].strip()]))
    return edges

The following code should now correctly create the graph.

In [4]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [5]:
def Search(graph, root):
    """Runs Search from vertex root in a graph. Neighboring nodes are processed in alphabetical order.
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.  
    Returns:
        A list of nodes in the order in which they were first visited.
    """    
    def Depth_First_Search(graph,root,node_visited):
        """Runs A helper function to run recursively the procedure for performing Depth First Search.
        Args:
            graph: the given graph, with nodes encoded as strings.
            root: the node from which to start the search. 
            node_visited: A dictionary that maintains the boolean value for whether a node is previously visited or not.
        Returns:
            A list of nodes in the order in which they were first visited.
        """
        
        # Setting the corresponding value for the root node in the visited array as True.
        node_visited[root]=True
        #An empty list which would store nodes in order they are visited.
        path=[]
        #Adding root to the list as it is the first node to be visited.
        path.append(root)
        
        #recursively running the procedure for all the neighboring nodes (Processed in an alphbetically sorted order)
        for neighbor in sorted(list(graph.neighbors(root))):
            #Visit the node only if it is not visited
            if(not(node_visited[neighbor])):
                #Setting the visited value for the specific node as True.
                node_visited[neighbor]=True
                #recursive call to Depth_First_Search making the neighbor node as root and appending the path from it to our initial path.
                path1=Depth_First_Search(graph,neighbor,node_visited)
                for node in path1:
                    path.append(node)
        return path
    
    #Extracting the list og nodes in the given graph
    nodes=list(graph.nodes)
    
    #initiating an empty dictionary and as initially all of them are not visited all the values corresponding to the nodes are set False.
    node_visited={}
    for node in nodes:
        node_visited[node]=False
    #Call to the helper function.
    path=Depth_First_Search(graph,root,node_visited)        
    return path  

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [6]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [7]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # A list to hold list of connected components
    components=[]
    
    #Getting all the nodes of the graph in alphabetically sorted order.
    nodes=sorted(list(graph.nodes()))
    
    #initiating an empty dictionary and as initially all of them are not visited all the values corresponding to the nodes are set False.
    node_visited={}
    for node in nodes:
        node_visited[node]=False
    
    #We need to find all connected components and hence The Search procedure is called untill all the nodes are visited.
    #The path returned by a Search call for one node represents the connected components for that node and hence are appended to the list of components
    for node in nodes:
        if(not(node_visited[node])):
            path=Search(graph,node)
            for element in path:
                node_visited[element]=True
            components.append(path)
    return components

We will check correctness of your code by verifying that your output list is identical to our solution.

In [8]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [9]:
component_sizes = [len(c) for c in character_interactions]
print("There are 12 connected components in the Iliad:{}".format(len(component_sizes) == 12))
print("The giant component has size 542:{}".format(max(component_sizes) == 542))
print("There are 5 isolated characters:{}".format(len([c for c in component_sizes if c == 1]) == 5))

There are 12 connected components in the Iliad:True
The giant component has size 542:True
There are 5 isolated characters:True
